# Сокеты. Клиент-сервер

Работа с сетью, сокеты
* Что такое сокеты?
* Зачем нужны сокеты?
* Программа клиент-сервер

Сокеты - это кросс-платформенный механизм для обмена данными между отдельными процессами.  
Эти процессы могут работать на разных серверах, могут быть написаны на разных языках.  
Программа на Python, которая использует механизм сокетов, осуществляет системные вызовы и взаимодействует с ядром операционной системой.

Для организации сетевого взаимодействия используются:
* Cервер, который изначально создает некое соединение и начинает "слушать" все запросы, которые поступают в него.
* Программа-клиент, которая присоединяется к серверу и отправляет ему данные.

## Создание сокета

### Сервер

In [ ]:
import socket

print("""Для начала работы с этим сервером, запустите в другой консоли на выбор клиента и отправьте данные:
    1) Запустите команду telnet 127.0.0.1 10001, а затем в сеансе отправьте любой текст
    2) Создайте клиентское соединение с сокетом сервера:
        with socket.create_connection(('127.0.0.1', 10001)) as sock:
            sock.sendall('ping from client'.encode('utf8'))
    3) Откройте браузер по адресу: http://127.0.0.1:10001
""")


# https://docs.python.org/3/library/socket.html
# http://lecturesnet.readthedocs.io/net/low-level/ipc/socket/intro.html
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('127.0.0.1', 10001)) # max port 65535. Метод bind() связывает сокет с IP-адресом и портом

# Метод listen() объявляет начало прослушивания входящих соединений. В параметре передается
# необязательный размер очереди входящих соединений, которые еще не обработаны, для которых еще не был
# вызван метод accept(). Если сервер не будет успевать принимать входящие соединения, то все соединения будут
# копится в этой очереди, и если она превысит это максимальное значение, то ОС выдаст ошибку
# connection refused для клиентской программы.
sock.listen(socket.SOMAXCONN)

# Вызывается метод accept() для того, чтобы начать принимать входящее клиентское соединение.
# Метод accept() блокирует поток, пока в сокете не появятся данные. Как только данные появляются,
# поток "просыпается" и метод accept() возвращает tuple из 2-х объектов:
#   1) объект типа <class 'socket.socket'> который является полнодуплексным каналом,
#      у которого доступны методы чтения и записи в этот канал:
#       <socket.socket fd=4, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM,
#                            proto=0, laddr=('127.0.0.1', 10001), raddr=('127.0.0.1', 44168)>
#   2) объект типа <class 'tuple'> который содержит адрес и порт:
#       ('127.0.0.1', 44168)
#
conn, addr = sock.accept() # Метод БЛОКИРУЕТ поток. Элементы из tuple присваиваются двум переменным conn и addr
print('Появилось соединение на сокете с клиентом:', addr)

# В этом бесконечном цикле вызываем чтение из полнодуплексного канала
while True:
    # Если данных нет, то основной поток (этот цикл while) блокируется методом recv() в ожидание
    # пока не появятся следующие данные в сокете от текущего соединения с клиентом.
    data = conn.recv(1024) # Метод БЛОКИРУЕТ поток. Данные из сокета считываются в бинарном формате!
    # Если клиент передал пустую строку или ключевое слово exit, то завершаем сеанс работы с клиентом
    if not data.decode('utf8').strip() or 'exit' in data.decode('utf8'):
        break
    # Выводим данные из сокета которые отправил клиент
    print(data.decode('utf8'), end='')

conn.close() # Закрываем соединение с клиентом
sock.close() # Закрываем сокет

Для начала работы с этим сервером, запустите в другой консоли на выбор клиента и отправьте данные:
    1) Запустите команду telnet 127.0.0.1 10001, а затем в сеансе отправьте любой текст
    2) Создайте клиентское соединение с сокетом сервера:
        with socket.create_connection(('127.0.0.1', 10001)) as sock:
            sock.sendall('ping from client'.encode('utf8'))
    3) Откройте браузер по адресу: http://127.0.0.1:10001

Появилось соединение на сокете с клиентом: ('127.0.0.1', 43246)
ffff
aaaa


Выполним соединение с сокетом:

In [2]:
%%bash

telnet 127.0.0.1 10001

Trying 127.0.0.1...
Connected to 127.0.0.1.
Escape character is '^]'.
ffff
aaaa
exit
Connection closed by foreign host.


### Клиент

In [ ]:
import socket

sock = socket.socket()
# Метод connect() заблокируется до тех пор, пока сервер со своей стороны не вызовет метод accept()
sock.connect(('127.0.0.1', 10001))
# После того, как метод connect() достучался до сервера, следом можно отправлять и получать данные с сервера.
# Пересылать данные по сети мы вынуждены в байтах, а не строках, поэтому кодируем строку в байты:
sock.sendall('ping 1'.encode('utf8'))
sock.close()

# Более короткая запись соединения:
# sock = socket.create_connection(('127.0.0.1', 10001))
# sock.sendall('ping 2'.encode('utf8'))
# sock.close()

## Создание сокета. Контекстный менеджер

### Сервер

In [ ]:
import socket

print("""Для начала работы с этим сервером, запустите в другой консоли на выбор клиента и отправьте данные:
    1) Запустите команду telnet 127.0.0.1 10001, а затем в сеансе отправьте любой текст
    2) Создайте клиентское соединение с сокетом сервера:
        with socket.create_connection(('127.0.0.1', 10001)) as sock:
            sock.sendall('ping from client'.encode('utf8'))
    3) Откройте браузер по адресу: http://127.0.0.1:10001
""")

# Используется контекстный менеджер, который автоматически закроет сокет:
with socket.socket() as sock:
    sock.bind(('', 10001))
    # В параметрах listen() можно использовать параметр socket.SOMAXCONN,
    # который задаст размер очереди ожиданий accept() на сокете.
    sock.listen()
    
    while True:
        print('Ожидание нового соединения клиента на сокете...')
        conn, addr = sock.accept() # Поток блокируется в ожидании нового соединения клиента с сокетом сервера
        print('Новое соединение с клиентом:', addr)
        
        with conn: # Используется контекстный менеджер, который автоматически закроет соединение с клиентом
            while True:
                data = conn.recv(1024).decode('utf8') # Метод блокирует поток, пока не появятся данные от клиента
                if not data.strip() or 'exit' in data:
                    break
                print(data)
        print('Соединение с клиентом закрыто')

Для начала работы с этим сервером, запустите в другой консоли на выбор клиента и отправьте данные:
    1) Запустите команду telnet 127.0.0.1 10001, а затем в сеансе отправьте любой текст
    2) Создайте клиентское соединение с сокетом сервера:
        with socket.create_connection(('127.0.0.1', 10001)) as sock:
            sock.sendall('ping from client'.encode('utf8'))
    3) Откройте браузер по адресу: http://127.0.0.1:10001

Ожидание нового соединения клиента на сокете...
Новое соединение с клиентом: ('127.0.0.1', 43420)
ping from client
Соединение с клиентом закрыто
Ожидание нового соединения клиента на сокете...


### Клиент

In [4]:
import socket

with socket.create_connection(('127.0.0.1', 10001)) as sock:
    sock.sendall('ping from client'.encode('utf8'))

## Таймауты и обработка сетевых ошибок

* Connect timeout и read timeout, в чем разница?
* Обработка ошибок

### Сервер

In [ ]:
import socket

with socket.socket() as sock:
    sock.bind(('', 10001))
    sock.listen() # Открываем сокет в ОС на прослушивание запросов
    
    while True:
        print('Ожидание нового соединения клиента на сокете')
        # Метод блокирующий поток, ожидающий соединения:
        conn, addr = sock.accept()
        # Если данные не поступают на сокет в течении 5 секунд,
        # то соединение с клиентом разрывается и сокет переходит в ожидание следующего подключения:
        conn.settimeout(5)
        print('Новое соединение клиента:', addr, 'с таймаутом:', conn.gettimeout())
        
        with conn:
            while True:
                try:
                    data = conn.recv(1024).decode('utf8')
                except socket.timeout:
                    print('Close connection by timeout')
                    break

                if not data.strip():
                    break

                print(data)
        
        print('Соединение с клиентом закрыто')

Ожидание нового соединения клиента на сокете
Новое соединение клиента: ('127.0.0.1', 43492) с таймаутом: 5.0
ping
Соединение с клиентом закрыто
Ожидание нового соединения клиента на сокете


### Клиент

In [3]:
import socket

# В параметрах соединения задается socket CONNECT timeout
with socket.create_connection(('127.0.0.1', 10001), 5) as sock:
    # Задается socket READ timeout (все операции с сокетом)
    sock.settimeout(2)
    try:
        sock.sendall('ping'.encode('utf8'))
    except socket.timeout:
        print('send data timeout')
    except socket.error as e:
        print('send data error:', e)

## Обработка нескольких соединений. Потоки

* Как обработать несколько соединений одновременно?
* Что использовать, процессы или потоки?
* Рассмотрим примеры обработки сетевых запросов

### Сервер

In [ ]:
import socket
import threading

def process_request(conn, addr):
    with conn:
        while True:
            # Метод recv() блокирует поток после получения данных от клиента в ожидание следующих данных:
            data = conn.recv(1024).decode('utf8')
            if not data.strip():
                break
            print(data, end='')


with socket.socket() as sock:
    sock.bind(('', 10001))
    sock.listen()
    
    while True:
        print('Ожидание нового соединения клиента на сокете')
        # Метод блокирующий поток, ожидающий соединений.
        conn, addr = sock.accept()
        print('Новое соединение клиента:', addr)
        # Создаем поток ОС
        th = threading.Thread(target=process_request, args=(conn, addr,))
        th.start() # Запускается поток обработки текущего соединения, а main поток идет дальше

        # В этом месте, основной поток продолжает исполнение и переходит на новую итерацию цикла while для ожидания
        # новых соединений, а запущенный поток обрабатывает существующее соединение.

## Обработка нескольких соединений. Потоки и процессы одновременно

### Сервер

In [ ]:
import os
import socket
import threading
import multiprocessing


def process_request(conn, addr):
    with conn:
        while True:
            data = conn.recv(1024).decode('utf8')
            if not data.strip():
                break
            print(data, end='')

def worker(sock):
    while True:
        print('Ожидание нового соединения клиента на сокете. PID:', os.getpid())
        conn, addr = sock.accept() # Здесь происходит блокирование до появления новых данных в сокете.
        print('Новое соединение клиента:', addr)
        th = threading.Thread(target=process_request, args=(conn, addr,))
        th.start()
        # А в этом месте, основной поток переходит на новую итерацию цикла while для ожидания
        # новых соединений, а запущенный поток обрабатывает уже существующее соединение.


with socket.socket() as sock:
    sock.bind(('', 10001))
    sock.listen()

    # Пытаемся преодалеть ограничение GIL на потоки с помощью процессов.
    # Генерируется список из 3-х объектов Process которые будут принимать соединения на сокете
    workers_list = [multiprocessing.Process(target=worker, args=(sock,)) for _ in range(3)]

    for w in workers_list:
        w.start() # Запускаем процесс
        w.join()  # и запускаем ожидание главным процессом завершение дочерних процессов